In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ../../

/home/tejomay/cgpos


In [3]:
import random 

import torch
import torch.nn as nn

from src.model import Transformer
import src.config as cfg
from src.util import read_pkl, encode, decode, get_batch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
train_size = 0.98
n_chunks = 500
random_seed = 20
unc_rate = 0.01

In [5]:
data = read_pkl(cfg.pt_syl)
vocab = ["<UNK>"] + sorted(set(data))
data = [d if random.random() > unc_rate else "<UNK>" for d in data]
vocab_size = len(vocab)
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}

In [19]:
d = defaultdict(lambda: 7, {4: 6, 7: 8})

In [24]:
d

defaultdict(<function __main__.<lambda>()>, {4: 6, 7: 8, 10: 7})

In [7]:
# Train and test split
tokens = torch.tensor(encode(stoi, data), dtype=torch.long)
chunks = torch.split(tokens, len(data) // (n_chunks - 1))
l = [1] * int(n_chunks * train_size) + [0] * int(n_chunks * (1 - train_size))
random.shuffle(l)
train_data = torch.cat([chunks[i] for i, v in enumerate(l) if v])
val_data = torch.cat([chunks[i] for i, v in enumerate(l) if not v])

In [8]:
x, y = get_batch(train_data, 256, 64, "cpu")

In [9]:
model = Transformer(
    vocab_size=16970, 
    block_size=256, 
    emb_size=512, 
    n_layer=6, 
    n_head=8, 
    dropout=0.6, 
    device=device
)
model.load_state_dict(torch.load('wts/2023-10-07_18-05-31.pth', map_location=torch.device(device)))
m = model.to(device)

In [10]:
model(x, None)

IndexError: index out of range in self

In [35]:
model

Transformer(
  (tok_emb_table): Embedding(16970, 512)
  (pos_emb_table): Embedding(256, 512)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=512, out_features=64, bias=False)
            (query): Linear(in_features=512, out_features=64, bias=False)
            (value): Linear(in_features=512, out_features=64, bias=False)
            (dropout): Dropout(p=0.6, inplace=False)
          )
        )
        (proj): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.6, inplace=False)
      )
      (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
          

In [49]:
model.blocks[0].ffwd.net[0]

Linear(in_features=512, out_features=2048, bias=True)

In [34]:
model.lm_head = nn.Linear

Linear(in_features=512, out_features=16970, bias=True)